# Imports

In [ ]:
import os
import json
import re
from pathlib import Path
import pandas as pd
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


'/content/drive/MyDrive/TCC/workspace/datasets'

In [ ]:
def create_prompt(row):
  # SYSTEM = "Você é um assistente especializado em analisar e identificar notícias falsas ou verdadeiras."
  # <|start_header_id|>system<|end_header_id|>{SYSTEM}<|eot_id|>
  INPUT = f'''Dada a notícia abaixo, classifique-a como "falsa" ou "verdadeira": {row["preprocessed_news"]}'''
  OUTPUT = row["label"]

  return f'''<|begin_of_text|><|start_header_id|>user<|end_header_id|>{INPUT}<|eot_id|>
  <|start_header_id|>assistant<|end_header_id|>{OUTPUT}<|eot_id|>'''

# Base line websites

## English

In [ ]:
realNews_path = os.path.join(DATASET_DIR, "realNewsWebsites.json")
with open(realNews_path) as file:
    real_website_urls = json.load(file)

real_website_urls

## Brazil

In [ ]:
'''
# The script below was used to create a csv from txt files, since it's to expansive to upload all txt files in google drive. Need to download true-meta-infomation folder
# from Fakebr corpus and run locally this script
import pandas as pd
def convert_meta_txt_to_csv(folder, output_file_name):
    columns = "author,link,category,date of publication,number of tokens,number of words without punctuation,number of types,number of links inside the news,number of words in upper case,number of verbs,number of subjuntive and imperative verbs,number of nouns,number of adjectives,number of adverbs,number of modal verbs (mainly auxiliary verbs),number of singular first and second personal pronouns,number of plural first personal pronouns,number of pronouns,pausality,number of characters,average sentence length,average word length,percentage of news with speeling errors,emotiveness,diversity"
    columns = columns.split(",")
    all_txt_files = os.listdir(folder)
    df = pd.DataFrame(columns=columns)
    for txt_name in all_txt_files:
        txt_file = f"{true_meta_data}/{txt_name}"
        with open(txt_file, encoding="utf8") as file:
            lines = file.readlines()

        new_row = dict(zip(columns,lines))
        df.loc[len(df)] = new_row

    file_name = f"{os.getcwd()}/{output_file_name}.csv"
    df.to_csv(file_name, index=False)

folder = f"{DATASET_DIR}/Fakebr/full_texts/true-meta-information"
output_file_name = "true_meta_data"
convert_meta_txt_to_csv(folder, output_file_name)
'''

import pandas as pd

custom_br_websites = '''g1.globo.com
folha.uol.com.br
oglobo.globo.com
estadao.com.br
noticias.uol.com.br
terra.com.br
r7.com
veja.abril.com.br
epoca.globo.com
valor.globo.com
correiobraziliense.com.br
jb.com.br
g1.globo.com/jornal-nacional
opovo.com.br
gazetadopovo.com.br
extra.globo.com
cartacapital.com.br
brasil247.com
redebrasilatual.com.br
em.com.br
agazeta.com.br
jovempan.com.br
band.uol.com.br/noticias
exame.com
folhape.com.br
bbc.com/portuguese
cnnbrasil.com.br
metropoles.com
istoe.com.br
revistaforum.com.br
jornaldebrasilia.com.br
gazetaonline.com.br
noticiasaominuto.com.br
ultimosegundo.ig.com.br
jornaldocomercio.com
brasildefato.com.br
poder360.com.br
congressoemfoco.uol.com.br
nexojornal.com.br
gazetaesportiva.com
placar.abril.com.br
lance.com.br
infomoney.com.br
agazetadoacre.com
www.gazetape.com
folhadelondrina.com.br
oprogressonet.com
jornaldotocantins.com.br
folhabv.com.br
acritica.com'''.split('\n')

def get_all_domains(file):

    csv = pd.read_csv(file)
    links = list(csv["link"])

    all_domains = set()
    for link in links:
        domain = link.split("/")[2]
        all_domains.add(domain)

    return all_domains

true_meta_data = f"{DATASET_DIR}/Fakebr/true_meta_data.csv"
domains = get_all_domains(true_meta_data)
for new_domain in custom_br_websites:
    domains.add(new_domain)

print(domains, len(domains))
domains_txt_file =  f"{DATASET_DIR}/realbrsites.txt"
with open(domains_txt_file, "w") as file:
  for domain in domains:
    file.write(f"{domain}\n")

{'noticiasaominuto.com.br', 'gazetaesportiva.com', 'folhabv.com.br', 'acervo.estadao.com.br', 'g1.globo.com/jornal-nacional', 'emais.estadao.com.br', 'jovempan.com.br', 'veja.abril.com.br', 'exame.com', 'jornaldocomercio.com', 'g1.globo.com', 'lance.com.br', 'infomoney.com.br', 'brasil.estadao.com.br', 'cnnbrasil.com.br', 'ultimosegundo.ig.com.br', 'politica.estadao.com.br', 'brasildefato.com.br', 'esportes.estadao.com.br', 'gazetadopovo.com.br', 'extra.globo.com', 'r7.com', 'link.estadao.com.br', 'oprogressonet.com', 'epoca.globo.com', 'correiobraziliense.com.br', 'jornaldotocantins.com.br', 'alias.estadao.com.br', 'jornaldebrasilia.com.br', 'www.estadao.com.br', 'agazetadoacre.com', 'sustentabilidade.estadao.com.br', 'terra.com.br', 'datafolha.folha.uol.com.br', 'saude.estadao.com.br', 'brasil247.com', 'www.territorioeldorado.limao.com.br', 'redebrasilatual.com.br', 'cultura.estadao.com.br', 'agazeta.com.br', 'band.uol.com.br/noticias', 'f5.folha.uol.com.br', 'noticias.uol.com.br', '

# Brasil Portuguese Dataset (Fake.br-Corpus)

## full text version

In [ ]:
'''
# The script below was used to create a csv from txt files, since it's to expansive to upload all txt files in google drive. Need to download true-meta-infomation folder
# from Fakebr corpus and run locally this script
import pandas as pd
def convert_txt_to_df(folder, label):
    columns = ["label", "text"]
    all_txt_files = os.listdir(folder)
    df = pd.DataFrame(columns=columns)
    for txt_name in all_txt_files:
        txt_file = f"{true_meta_data}/{txt_name}"
        with open(txt_file, encoding="utf8") as file:
            lines = file.readlines()

        truncated_str = "".join([string.strip("\n") for string in lines])
        new_row = [label, truncated_str]
        df.loc[len(df)] = new_row

    return df

true_folder = f"{DATASET_DIR}/Fakebr/full_texts/true"
fake_folder = f"{DATASET_DIR}/Fakebr/full_texts/fake"
true_df = convert_txt_to_df(true_folder, label=1)
fake_df = convert_txt_to_df(fake_folder, label=0)

df = pd.concat([true_df, fake_df])
df.to_csv(f"{DATASET_DIR}/Fakebr/full_text_fakebrcorpus.csv",index=False)
'''

In [ ]:
fakebrcorpus_full = pd.read_csv(f"{DATASET_DIR}/Fakebr/full_text_fakebrcorpus.csv")
fakebrcorpus_full

,label,text
0,1,﻿O Podemos decidiu expulsar o deputado federa...
1,1,"Bolsonaro é um liberal completo, diz president..."
2,1,Ministro do STF libera Andrea Neves de prisão ...
3,1,"Apesar da abundância, cresce preocupação com p..."
4,1,"Por que Harvard e MIT levarão Dilma, Moro e Su..."
...,...,...
7195,0,"FT: ""O julgamento de um populista que usou din..."
7196,0,Nota fiscal emitida pela Petrobras mostra que ...
7197,0,"Estão estancando a sangria da ""Lava-Jato"" bem ..."
7198,0,"(RJ) Cidadão finge estar possuído pela ""pomba ..."


# LIAR Dataset ("Liar, Liar Pants on Fire": A New Benchmark Dataset for Fake News Detection)

In [ ]:
liar_train = pd.read_csv(f'{DATASET_DIR}/LIAR/train.tsv', sep='\t', header=None)
liar_test = pd.read_csv(f'{DATASET_DIR}/LIAR/test.tsv', sep='\t', header=None)
liar_valid = pd.read_csv(f'{DATASET_DIR}/LIAR/valid.tsv', sep='\t', header=None)

# liar_train.columns = ["label", "text"]
# liar_test.columns = ["label", "text"]
# liar_valid.columns = ["label", "text"]

LIAR = pd.concat([liar_train, liar_test, liar_valid])
LIAR = LIAR[LIAR[1].isin(['true','false'])]
LIAR

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
5,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
12,5947.json,false,When Mitt Romney was governor of Massachusetts...,"history,state-budget",mitt-romney,Former governor,Massachusetts,republican,34.0,32.0,58.0,33.0,19.0,an interview with CBN News
16,620.json,true,McCain opposed a requirement that the governme...,federal-budget,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a radio ad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,1431.json,false,We've excluded lobbyists from policymaking jobs.,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,his State of the Union address
1271,5885.json,false,Says George Flinn has been a no show at schedu...,campaign-advertising,charlotte-bergmann,Candidate,Tennessee,republican,0.0,2.0,0.0,0.0,1.0,in a press release calling for debates.
1275,278.json,false,"I supported (the surge), I argued for it. I'm ...",iraq,john-mccain,U.S. senator,Arizona,republican,31.0,39.0,31.0,37.0,8.0,"a debate in Myrtle Beach, S.C."
1281,401.json,true,"John McCain and George Bush have ""absolutely n...",health-care,campaign-defend-america,NaN,"Washington, D.C.",none,0.0,1.0,0.0,2.0,0.0,a television ad


In [ ]:
LIAR[LIAR[1]=='true']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
5,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
16,620.json,true,McCain opposed a requirement that the governme...,federal-budget,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a radio ad
30,13237.json,true,Says Paul Ryan is still endorsing Trump.,"candidates-biography,elections,campaign-advert...",hillary-clinton,Presidential candidate,New York,democrat,40.0,29.0,69.0,76.0,7.0,a tweet
35,7550.json,true,We have a federal government that thinks they ...,"consumer-safety,corrections-and-updates,govern...",ted-cruz,Senator,Texas,republican,36.0,33.0,15.0,19.0,8.0,a speech at CPAC
36,9018.json,true,Austin is a city that has basically doubled in...,"city-government,population",lee-leffingwell,Austin mayor,Texas,democrat,1.0,0.0,4.0,1.0,0.0,"his ""State of the City"" address"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258,3632.json,true,Large majorities of the public oppose major ch...,"federal-budget,medicare,polls,pundits,taxes",paul-krugman,New York Times columnist,NaN,none,0.0,2.0,3.0,4.0,0.0,a newspaper column
1260,7983.json,true,Says spending in the fiscal 2009 budget was lo...,"state-budget,state-finances",barbara-buono,State Senator,New Jersey,democrat,3.0,1.0,4.0,4.0,1.0,a state Senate budget hearing
1263,3984.json,true,The U.S. is spending one out of every six Defe...,"afghanistan,federal-budget,military",jon-huntsman,NaN,Utah,republican,5.0,1.0,5.0,4.0,1.0,"an interview on ""Morning Joe."""
1265,1825.json,true,"The Republican Party ""fought very hard in the ...","civil-rights,elections,history,legal-issues,ab...",michael-steele,chairman of the Republican National Committee,Maryland,republican,0.0,1.0,1.0,0.0,2.0,an interview on ABC's This Week


In [ ]:
liar = pd.read_csv(f'{DATASET_DIR}/LIAR/train.tsv', sep='\t', header=None)
liar

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report"""
10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview
10237,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate
10238,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...


In [ ]:
liar_with_columns = liar[[1, 2]]
liar_with_columns.columns = ["label", "text"]
liar_with_columns

,label,text
0,false,Says the Annies List political group supports ...
1,half-true,When did the decline of coal start? It started...
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo..."
3,false,Health care reform legislation is likely to ma...
4,half-true,The economic turnaround started at the end of ...
...,...,...
10235,mostly-true,There are a larger number of shark attacks in ...
10236,mostly-true,Democrats have now become the party of the [At...
10237,half-true,Says an alternative to Social Security that op...
10238,false,On lifting the U.S. Cuban embargo and allowing...


In [ ]:
liar_train_v2 = liar_with_columns[(liar_with_columns["label"] == "true") | (liar_with_columns["label"] == "false")]
liar_train_v2.to_csv(f'{DATASET_DIR}/LIAR/liar_train_v2.csv',
                      header=["label", "text"], index=False)

In [ ]:
liar_train_v2[liar_train_v2['label']=='false']

,label,text
0,false,Says the Annies List political group supports ...
3,false,Health care reform legislation is likely to ma...
12,false,When Mitt Romney was governor of Massachusetts...
20,false,Women and men both are making less when you ad...
25,false,I dont know who (Jonathan Gruber) is.
...,...,...
10214,false,"Under last years health care reform, a bunch o..."
10216,false,There has been no net global warming for over ...
10218,false,This race will be the Democrats top target thi...
10221,false,"As a result of Obamacare, California seniors f..."


In [ ]:
liar_test = pd.read_csv(f'{DATASET_DIR}/LIAR/test.tsv', sep='\t', header=None)
liar_test_with_columns = liar_test[[1, 2]]
liar_test_with_columns.columns = ["label", "text"]
liar_test_v2 = liar_test_with_columns[(liar_test_with_columns["label"] == "true") | (liar_test_with_columns["label"] == "false")]
liar_test_v2.to_csv(f'{DATASET_DIR}/LIAR/liar_test_v2.csv',
                      header=["label", "text"], index=False)
liar_test_v2

,label,text
0,true,Building a wall on the U.S.-Mexico border will...
1,false,Wisconsin is on pace to double the number of l...
2,false,Says John McCain has done nothing to help the ...
5,true,Over the past five years the federal governmen...
6,true,Says that Tennessee law requires that schools ...
...,...,...
1247,true,Says Barack Obama promised to halve the defici...
1253,true,I am the only senator who turned down the stat...
1255,false,There is no system to vet refugees from the Mi...
1260,false,I think its seven or eight of the California s...


## final version

In [ ]:
liar_full = pd.read_csv(f'{DATASET_DIR}/LIAR/liar_train.csv')
liar_full["label"] = liar_full["label"].replace({False: 0, True: 1})
liar_full

<ipython-input-6-dd7c78a85d38>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  liar_full["label"] = liar_full["label"].replace({False: 0, True: 1})


,label,text
0,0,Says the Annies List political group supports ...
1,0,Health care reform legislation is likely to ma...
2,1,The Chicago Bears have had more starting quart...
3,0,When Mitt Romney was governor of Massachusetts...
4,1,McCain opposed a requirement that the governme...
...,...,...
3666,0,There has been no net global warming for over ...
3667,0,This race will be the Democrats top target thi...
3668,0,"As a result of Obamacare, California seniors f..."
3669,1,"For the first time since the Korean War, total..."


In [ ]:
liar_full.to_csv(f'{DATASET_DIR}/LIAR/liar_full.csv', index=False)

In [ ]:
liar_full = pd.read_csv(f'{DATASET_DIR}/LIAR/liar_full.csv')
liar_full

,label,text
0,0,Says the Annies List political group supports ...
1,0,Health care reform legislation is likely to ma...
2,1,The Chicago Bears have had more starting quart...
3,0,When Mitt Romney was governor of Massachusetts...
4,1,McCain opposed a requirement that the governme...
...,...,...
7337,0,There has been no net global warming for over ...
7338,0,This race will be the Democrats top target thi...
7339,0,"As a result of Obamacare, California seniors f..."
7340,1,"For the first time since the Korean War, total..."


In [ ]:
len(liar_full[liar_full['label'] == 0]), len(liar_full[liar_full['label'] == 1])

(3990, 3352)

In [ ]:
len(liar_full[liar_full['label'] == 0])